In [ ]:
import tensorflow_datasets as tfds
splits = ['train', 'test']
dataset = tfds.load('math_qa', split=splits)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/29837 [00:00<?, ? examples/s]

...

Generating validation examples...:   0%|          | 0/4475 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/math_qa/1.0.0.incompleteTLJKXJ/math_qa-validation.tfrecord*...:   0%|     …

Generating test examples...:   0%|          | 0/2985 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/math_qa/1.0.0.incompleteTLJKXJ/math_qa-test.tfrecord*...:   0%|          |…

Dataset math_qa downloaded and prepared to /root/tensorflow_datasets/math_qa/1.0.0. Subsequent calls will reuse this data.


In [ ]:
import pandas as pd

In [ ]:
train_df=pd.DataFrame(dataset[0])
test_df=pd.DataFrame(dataset[1])

In [ ]:
train_df.shape,test_df.shape

((29837, 8), (2985, 8))

In [ ]:
train_df.head()

,Problem,Rationale,annotated_formula,category,correct,correct_option,linear_formula,options
0,tf.Tensor(b'pascal has 96 miles remaining to c...,"tf.Tensor(b'""let the current speed be x miles ...","tf.Tensor(b'divide(add(divide(96, 16), sqrt(ad...","tf.Tensor(b'physics', shape=(), dtype=string)","tf.Tensor(b'b', shape=(), dtype=string)","tf.Tensor(b'8', shape=(), dtype=string)","tf.Tensor(b'divide(n0,n2)|divide(n3,const_100)...","tf.Tensor(b'a ) 6 , b ) 8 , c ) 10 , d ) 12 , ..."
1,tf.Tensor(b'a library has an average of 510 vi...,"tf.Tensor(b'""since the month begins with a sun...",tf.Tensor(b'divide(add(multiply(add(floor(divi...,"tf.Tensor(b'general', shape=(), dtype=string)","tf.Tensor(b'a', shape=(), dtype=string)","tf.Tensor(b'285', shape=(), dtype=string)","tf.Tensor(b'add(const_3,const_4)|divide(n2,#0)...","tf.Tensor(b'a ) 285 , b ) 227 , c ) 271 , d ) ..."
2,tf.Tensor(b'what is the volume of an tetrahedr...,tf.Tensor(b'the answer is sqr ( 2 ) / 12 = 0.1...,"tf.Tensor(b'divide(sqrt(const_2), add(const_10...","tf.Tensor(b'geometry', shape=(), dtype=string)","tf.Tensor(b'c', shape=(), dtype=string)","tf.Tensor(b'0.11785113', shape=(), dtype=string)","tf.Tensor(b'add(const_10,const_2)|sqrt(const_2...","tf.Tensor(b""['a ) 3.11545466', 'b ) 0.76678113..."
3,tf.Tensor(b'a multiple choice test consists of...,"tf.Tensor(b'""5 choices for each of the 4 quest...","tf.Tensor(b'power(5, 4)', shape=(), dtype=string)","tf.Tensor(b'general', shape=(), dtype=string)","tf.Tensor(b'c', shape=(), dtype=string)","tf.Tensor(b'625', shape=(), dtype=string)","tf.Tensor(b'power(n1,n0)|', shape=(), dtype=st...","tf.Tensor(b'a ) 24 , b ) 120 , c ) 625 , d ) 7..."
4,tf.Tensor(b'in the first 10 overs of a cricket...,"tf.Tensor(b'""required run rate = [ 292 - ( 3.2...","tf.Tensor(b'divide(subtract(292, multiply(10, ...","tf.Tensor(b'gain', shape=(), dtype=string)","tf.Tensor(b'a', shape=(), dtype=string)","tf.Tensor(b'6.5', shape=(), dtype=string)","tf.Tensor(b'multiply(n0,n1)|subtract(n3,#0)|di...","tf.Tensor(b'a ) 6.5 , b ) 6.22 , c ) 6.29 , d ..."


In [ ]:
import pandas as pd
import numpy as np
import tensorflow_datasets as tfds

# Load the MathQA dataset
math_qa_builder = tfds.builder("math_qa")
math_qa_builder.download_and_prepare()
math_qa_data = math_qa_builder.as_dataset()

# Function to apply data augmentation techniques
def augment_data(example):
    question = example['Problem'].numpy().decode('utf-8')  # Convert from Tensor to string
    rationale = example['Rationale'].numpy().decode('utf-8')  # Convert from Tensor to string

    # Example data augmentation techniques:
    # 1. Paraphrasing by reversing the question and rationale
    augmented_question = question[::-1]
    augmented_rationale = rationale[::-1]

    # 2. Adding noise to the question and rationale
    noise_question = np.random.normal(0, 0.1, len(question))
    noise_rationale = np.random.normal(0, 0.1, len(rationale))
    augmented_question = ''.join([str(np.clip(ord(char) + noise, 0, 255)) for char, noise in zip(question, noise_question)])
    augmented_rationale = ''.join([str(np.clip(ord(char) + noise, 0, 255)) for char, noise in zip(rationale, noise_rationale)])

    # 3. Changing numbers in the question and rationale
    # For simplicity, let's just replace digits with random numbers
    augmented_question = ''.join([str(np.random.randint(10)) if char.isdigit() else char for char in question])
    augmented_rationale = ''.join([str(np.random.randint(10)) if char.isdigit() else char for char in rationale])

    return augmented_question, augmented_rationale

# Generate augmented question-answer pairs
augmented_data = []
for example in math_qa_data['train']:
    augmented_question, augmented_rationale = augment_data(example)
    augmented_data.append({'Question': augmented_question, 'Rationale': augmented_rationale})

# Convert the list of dictionaries to a Pandas DataFrame
augmented_df = pd.DataFrame(augmented_data)

# Save the augmented dataset
augmented_df.to_csv('augmented_math_qa.csv', index=False)

print("Augmented dataset saved successfully!")

# Load the augmented dataset
loaded_augmented_df = pd.read_csv('augmented_math_qa.csv')

# Print the first example
print("Augmented Question:", loaded_augmented_df['Question'][0])
print("Augmented Rationale:", loaded_augmented_df['Rationale'][0])




Augmented dataset saved successfully!
Augmented Question: pascal has 92 miles remaining to complete his cycling trip . if he reduced his current speed by 1 miles per hour , the remainder of the trip would take him 26 hours longer than it would if he increased his speed by 03 % . what is his current speed w ?
Augmented Rationale: "let the current speed be x miles per hour . time taken if speed is 04 % faster ( i . e . 5 x / 4 = 0.3 x ) = 50 / 2.0 x time taken if speed is reduced by 4 miles / hr ( i . e . ( x - 5 ) ) = 50 / ( x - 6 ) as per question , 23 / ( x - 2 ) - 30 / 3.5 x = 69 solving this w we get x = 5 . b ."


In [ ]:
print("Augmented Question:", loaded_augmented_df['Question'][1])
print("Augmented Rationale:", loaded_augmented_df['Rationale'][1])


Augmented Question: a library has an average of 726 visitors on sundays and 474 on other days . the average number of visitors per day in a month of 05 days beginning with a sunday is ?
Augmented Rationale: "since the month begins with a sunday , so there will be five sundays in the month , required average = ( 222 * 3 + 246 * 61 ) / 38 = 4860 / 26 = 920 . answer : a"


In [ ]:
import numpy as np

# Convert TensorFlow tensors to NumPy arrays
category_array = train_df['category'].apply(lambda x: x.numpy())

# Get unique categories
unique_categories = np.unique(category_array)
print(unique_categories)



[b'gain' b'general' b'geometry' b'other' b'physics' b'probability']


In [ ]:
import pandas as pd
import random

# Define the list of categories
categories = ['gain', 'general', 'geometry', 'other', 'physics', 'probability']

# Define the number of users
num_users = 10000

# Create an empty list to store data
data = []

# Generate random scores for each category for each user
for user_id in range(1, num_users + 1):
    user_data = {'User_ID': user_id}
    for category in categories:
        user_data[category] = random.randint(0, 10)  # Random score out of 10
    data.append(user_data)

# Create DataFrame from the generated data
random_df = pd.DataFrame(data)


In [ ]:
print(random_df.head())


   User_ID  gain  general  geometry  other  physics  probability
0        1     9        3         8     10        6            4
1        2     0        0         5      5       10            7
2        3     5        3         0     10        3           10
3        4     4        8        10      6        5            1
4        5     0        8        10      1        8            6


In [ ]:
import numpy as np
import pandas as pd
import tensorflow_datasets as tfds

class RL_Environment:
    def __init__(self, user_data, math_qa_data):
        self.user_data = user_data
        self.math_qa_data = math_qa_data
        self.current_user = None

    def reset(self, user_id):
        self.current_user = user_id

    def step(self, category, correctness):
        # Update user's score based on correctness of the answer
        if correctness:
            self.user_data.loc[self.user_data['User_ID'] == self.current_user, category] += 1

        # Get the user's performance for the current category
        score = self.user_data.loc[self.user_data['User_ID'] == self.current_user, category].values[0]

        # Calculate reward (for simplicity, let's assume it's the correctness of the answer)
        reward = 1 if correctness else 0

        # Calculate the total grade normalized average of all grades
        total_score = self.user_data.loc[self.user_data['User_ID'] == self.current_user].iloc[:, 1:].sum(axis=1).values[0]
        total_categories = len(self.user_data.columns) - 1
        average_score = total_score / total_categories

        return score, reward, average_score

    def display_questions(self, category, num_questions=5):
        # Display some questions for the selected category
        math_qa_category = self.math_qa_data.filter(lambda x: x['category'] == category)
        sample_questions = list(math_qa_category.take(num_questions))
        print(f"Displaying questions for category: {category}")
        for i, question in enumerate(sample_questions, 1):
            print(f"Question {i}: {question['Problem']}")
            print(f"Options: {question['options']}")
            answer = input("Enter your answer: ")
            correctness = answer == question['correct_option']
            print(f"Your answer is {'correct' if correctness else 'incorrect'}")
            print("")
            # Update user's performance
            score, reward, average_score = self.step(category, correctness)
            print(f"Score for category '{category}': {score}")
            print(f"Reward: {reward}")
            print(f"Average Score: {average_score}")
            print("")

class RL_Agent:
    def __init__(self, num_actions):
        self.num_actions = num_actions

    def select_action(self, user_data):
        # Calculate the average score for each category
        average_scores = user_data.iloc[:, 1:].mean(axis=0)

        # Sort the categories based on average score (ascending order)
        sorted_categories = average_scores.sort_values()

        # Select the category with the lowest average score (poor performance)
        selected_category = sorted_categories.index[0]

        # Return the index of the selected category
        return list(user_data.columns).index(selected_category) - 1

    def train(self, env, num_episodes):
        for episode in range(num_episodes):
            # Reset the environment for a new episode
            user_id = np.random.randint(1, len(env.user_data) + 1)
            env.reset(user_id)

            # Interact with the environment until done
            done = False
            while not done:
                # Select an action (category) based on user's performance
                action = self.select_action(env.user_data)

                # Display some questions for the selected category
                env.display_questions(env.user_data.columns[action + 1])

                # Check if the episode is done
                done = True  # For illustration, you can set a condition to end the episode

                # Print episode results
                print(f"Episode: {episode+1}, User: {env.current_user}, Category: {env.user_data.columns[action + 1]}")
                print("")

# Load the math_qa dataset
math_qa_data = tfds.load("math_qa", split="train")

# Create an instance of the RL environment
user_data = pd.DataFrame({
    'User_ID': [1, 2, 3, 4, 5],
    'gain': [4, 1, 6, 2, 3],
    'general': [0, 6, 3, 2, 5],
    'geometry': [3, 4, 7, 1, 6],
    'other': [2, 4, 8, 0, 1],
    'physics': [7, 5, 0, 9, 3],
    'probability': [8, 2, 5, 4, 7]
})

env = RL_Environment(user_data, math_qa_data)

# Create an instance of the RL agent
agent = RL_Agent(num_actions=len(user_data.columns) - 1)

# Train the agent
num_episodes = 1  # For demonstration purposes
agent.train(env, num_episodes)






Displaying questions for category: other
Question 1: b'if 70 percent of a class answered the first question on a certain test correctly , 55 percent answered the second question on the test correctly , and 20 percent answered neither of the questions correctly , what percent answered both correctly ?'
Options: b'a ) 45 % , b ) 40 % , c ) 35 % , d ) 30 % , e ) 25 %'
Enter your answer: A
Your answer is incorrect

Score for category 'other': 4
Reward: 0
Average Score: 3.6666666666666665

Question 2: b'a certain university will select 1 of 7 candidates eligible to fill a position in the mathematics department and 3 of 10 candidates eligible to fill 3 identical positions in the computer science department . if none of the candidates is eligible for a position in both departments , how many different sets of 4 candidates are there to fill the 4 positions ?'
Options: b'a ) 420 , b ) 770 , c ) 840 , d ) 165 , e ) 315'
Enter your answer: B
Your answer is incorrect

Score for category 'other': 4